<h2>Lesson c: Evaluate our human-driven hint search algorithm

We want to embrace our current <i>scan</i> implementation with simple domain monitoring. 

In [ ]:
#load """Domain.fsx"""
#load """Cities\Toronto.fsx"""

open Domain

Toronto.license.info |> display

let addresses , districts, streets, summary = 
    setup Toronto.data Area.Neighbourhoods (Limit.Streets 500) 

let positions = addresses |> Array.map (fun x -> x.Pos) |> Array.toList

let indexedDistricts = districts |> Array.indexed

Open Government Licence – Toronto
(https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-licence/)
Attribution: Contains information licensed under the Open Government Licence – Toronto

Now we need to adapt our <i>hintSearch</i> human-driven algorithm slightly.
To mark computation as avoid or not we should  appropriate annotation. We could use true/false for that but let's use more descriptive type:

In [ ]:
type HintResult = Hit | Missed

Apparently to apply that type we only have to modify the algorithm in to places.
Notice that <b>scan</b> usage hasn't change at all

In [ ]:

let hintSearch (_, lastHitAreaIndex) currentPosition  =
    let hintArea = indexedDistricts.[lastHitAreaIndex]
    let find = snd >> isPointWithinFeature currentPosition

    if find hintArea then Hit, lastHitAreaIndex
    else Missed, indexedDistricts |> Array.findIndex find

let h::positions = positions
let init = hintSearch (Missed,0) h

let addressesAreas = (init, positions) ||> List.scan hintSearch 

Figure out the number how often our suggestion succeded

In [ ]:
let hit = 
    addressesAreas 
    |> List.filter (fst >> (=) Hit) 
    |> List.length 
    |> float


hit * 100. / (List.length addressesAreas |> float) |> display


80.26589252136047

Hmmm, it looks like we avoided a lot of full collection searches !
